In [ ]:
print('starting imports')
import numpy as np
import holoviews as hv
from holoviews import dim
hv.extension('bokeh')
#hv.extension('matplotlib')
import random

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, spread
from holoviews.operation.datashader import rasterize, ResamplingOperation

POINTS_COUNT_TARGET = 20000
ITERATIONS_BETWEEN_DISPLAY_UPDATES=20  # try 10 or 100



In [ ]:
rasterize

In [ ]:
# this is triangle only, not polygons generally
def generate_triangle_points_2d(iterations):
    targets = [(400, 0), (0, 599), (799,599)]
    points = []
    point = (100,100)
    i = 1

    while i <= iterations:
        target = random.choice(targets)
        point = ((point[0]+target[0])/2.0 , (point[1]+target[1])/2.0)
        points.append(point)
        i = i + 1
    return points    


In [ ]:
#generate_gasket_points(3, 10)  
def make_holoviz_triangle_points_obj(iterations):
    return hv.Points(generate_triangle_points_2d(iterations))
#    scatter = hv.Scatter(points)  
#    scatter.opts(color='k', marker='s', size=.1)

In [ ]:
# works
triangle=hv.DynamicMap(make_holoviz_triangle_points_obj, kdims=['iterations'])
triangle.redim.range(iterations=(100,10000))

In [ ]:
# this cell is copied from SO; it's for finding the points for vertices of n-gons
import math

# This function gets just one pair of coordinates based on the angle theta
def get_circle_coord(theta, x_center, y_center, radius):
    x = radius * math.cos(theta) + x_center
    y = radius * math.sin(theta) + y_center
    return (x,y)

# This function gets all the pairs of coordinates
def get_all_circle_coords(x_center, y_center, radius, n_points):
    thetas = [i/n_points * math.tau for i in range(n_points)]
    circle_coords = [get_circle_coord(theta, x_center, y_center, radius) for theta in thetas]
    #circle_coords.append([x_center+100, y_center+100])
    return circle_coords



In [ ]:
# this is my cutting-edge thing MINUS datashader 2022-11-01
# general polygons
def generate_polygon_points_2d(polygon_order, iterations):
    #targets = [(400, 0), (0, 599), (799,599)]
    targets = get_all_circle_coords(x_center = 800, 
                                      y_center = 800,
                                      radius = 750,
                                      n_points = polygon_order)    
    points = []
    point = (100,100)
    i = 1

    while i <= iterations:
        target = random.choice(targets)
        point = ((point[0]+target[0])/2.0 , (point[1]+target[1])/2.0)
        points.append(point)
        i = i + 1
    return points    

def make_holoviz_polygon_points_obj(polygon_vertices, iterations):
    points_obj = hv.Points(generate_polygon_points_2d(polygon_vertices,iterations))
    points_obj.opts(color='k', size=.1)
    return points_obj

#    scatter = hv.Scatter(points)  
#    scatter.opts(color='k', marker='s', size=.1)

# works
sierpinksi_gasket_polygon_live_view=hv.DynamicMap(make_holoviz_polygon_points_obj, kdims=['polygon_vertices', 'iterations'])
sierpinksi_gasket_polygon_live_view.redim.range(polygon_vertices=(3, 50), iterations=(500,500000))

In [ ]:
# datashader!! 
# works
sierpinksi_gasket_polygon_live_view=hv.DynamicMap(make_holoviz_polygon_points_obj, kdims=['polygon_vertices', 'iterations'])
rasterize(sierpinksi_gasket_polygon_live_view.redim.range(polygon_vertices=(2, 50), iterations=(10000,1000000))).opts(plot=dict(width=800, height=800))

In [ ]:
# Using the second function to generate all the pairs of coordinates.
get_all_circle_coords(x_center = 400, 
                                      y_center = 400,
                                      radius = 350,
                                      n_points = 3)

In [ ]:
# general polygons
def generate_polygon_points_2d_experiment(polygon_order, iterations):
    #targets = [(400, 0), (0, 599), (799,599)]
    targets = [(750.0, 400.0),
 (225.00000000000006, 703.1088913245535),
 (224.99999999999986, 96.89110867544656)]
    points = []
    point = (100,100)
    i = 1

    while i <= iterations:
        target = random.choice(targets)
        point = ((point[0]+target[0])/2.0 , (point[1]+target[1])/2.0)
        points.append(point)
        i = i + 1
    return points    

#generate_gasket_points(3, 10)  
def make_holoviz_triangle_points_obj_experiment(iterations):
    points_obj = hv.Points(generate_polygon_points_2d_experiment(3,iterations))
    points_obj.opts(color='k', size=1)
    return points_obj
#    scatter = hv.Scatter(points)  
#    scatter.opts(color='k', marker='s', size=.1)

In [ ]:
# works  but obsoleted by the polygon stuff
triangle2=hv.DynamicMap(make_holoviz_triangle_points_obj_experiment, kdims=['iterations'])
triangle2.redim.range(iterations=(100,10000))

In [ ]:
# WIWL:
# well, not much has been submmitted to HN regarding Holoviz: https://hn.algolia.com/?q=holoviz
# maybe FaceTime with Dave and see if he thinks this is cool and worth a post.
# FUTURE DIRECTIONS: make this thing take a lambda for "next_target_point" which would maybe contain state about all (or some?) previous targets. Then you can try the full "Chaos Game" as described out there on the web.
# Aha! John D Cook and Peter Norvig posted some Chaos Game stuff: 
#.  https://www.johndcook.com/blog/2017/07/08/the-chaos-game-and-the-sierpinski-triangle/?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+TheEndeavour+%28The+Endeavour%29 
#.  https://github.com/norvig/pytudes/blob/main/ipynb/Sierpinski.ipynb
# Holoviews is really good for trying to find cool polygons for this -- in fact Peter doesn't mention that a 7-gon has some structure! So I've got that to offer.
#    And also I should look to see what people have done with the 3D version -- since I have a pretty cool Matplotlib 3D one in triangle.make.array.3d.py.
# And of course there's the above exploration with "next target" rules inspired by the "Chaos Game" Google search. Some aspects of those rules can be parameterized with a slider or some such, like "how many steps back do we disallow repeated targets". There could be others! Ex. move toward the target with some other fraction of the distance, not 1/2. 
#     Is there other Holoviz interactivity available?
# There are rules worth trying that aren't easily defined / adjusted with sliders. ()
# Can you interactively change the size of the dots? I guess so, because that's defined in the first function.
# OK. How about a measure of "how much structure is there in these points?" That would allow for some code to go digging .  I supposed you could use "distribution of distance to nearest 100 neighbors". But that would definitely not be invariant of iterations. Or I guess that "how fractal is this curve" problem that M and I puzzled on would be good. Take a grid and count squares with no points. Plot that count against grid square size. In a max entropy collection of points this should smoothly change. But with structure there should be cliffs.
 # Oh wow -- this is great: http://www.oftenpaper.net/sierpinski.htm   So it mentions that using the assigning to the 4 corners of a diamond the 4 basic DNA possibiliities, GATC, then, although the random sierpinski of this is just grey goo, actually chunks of real species DNA creates patterns! Interesting! 
#.   Idea: What about a 26-gon with each English letter assigned a vertex and made on document corpuses? 
#.      Well, I have to try this.
# Also I want to need to see what 3d shapes work -- and what they produce? I would guess that since 4 points produces sierpinski's pyramid, and 4 is one fewer than the count of dimensions, maybe 5 wouldn't work (because in 2D 4 vertices doesn't work) and 6, 7, 8 would work, (because analogous to 5,6,7).
# Oh! That person points out: 
#.  that of course you can just define a state transition table -- so this would be easier to parameterize.
#   you don't have to do proper polygons -- certainly you could try obtuse polygons. (Which of course fill )
# https://hardmath123.github.io/chaos-game-fractal-foliage.html  I like, but prob not very relevant.
#. need to google search 'publish jupyter notebook online animation' something like that
# IDK what Panel adds to Holoviz but it doesn't really matter because what I have is working fine https://panel.holoviz.org/
# what I really want, I guess just for internal point experimentation, is "add a vertex" and "move a vertex" functionality.
# Next up: datashader!! 




In [ ]:
pts = generate_polygon_points_2d_experiment(3, 1000)

